## Policy and Value Iteration in Python

In [1]:
import gym
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mdptoolbox
import mdptoolbox.example
from gym import wrappers

%matplotlib inline

## Environment Exploration

In [2]:
#create gym evironment
env = gym.envs.make("CartPole-v0")

[2018-10-25 02:37:12,039] Making new env: CartPole-v0


In [3]:
env.spec.max_episode_steps

200

In [4]:
env.spec.timestep_limit

200

In [5]:
env.action_space.n

2

In [6]:
env.observation_space

Box(4,)

## Model Exploration

"""A Markov Decision Problem.

    Let ``S`` = the number of states, and ``A`` = the number of acions.

    Parameters
    ----------
    transitions : array
        Transition probability matrices. These can be defined in a variety of
        ways. The simplest is a numpy array that has the shape ``(A, S, S)``,
        though there are other possibilities. It can be a tuple or list or
        numpy object array of length ``A``, where each element contains a numpy
        array or matrix that has the shape ``(S, S)``. This "list of matrices"
        form is useful when the transition matrices are sparse as
        ``scipy.sparse.csr_matrix`` matrices can be used. In summary, each
        action's transition matrix must be indexable like ``transitions[a]``
        where ``a`` ∈ {0, 1...A-1}, and ``transitions[a]`` returns an ``S`` ×
        ``S`` array-like object.
    reward : array
        Reward matrices or vectors. Like the transition matrices, these can
        also be defined in a variety of ways. Again the simplest is a numpy
        array that has the shape ``(S, A)``, ``(S,)`` or ``(A, S, S)``. A list
        of lists can be used, where each inner list has length ``S`` and the
        outer list has length ``A``. A list of numpy arrays is possible where
        each inner array can be of the shape ``(S,)``, ``(S, 1)``, ``(1, S)``
        or ``(S, S)``. Also ``scipy.sparse.csr_matrix`` can be used instead of
        numpy arrays. In addition, the outer list can be replaced by any object
        that can be indexed like ``reward[a]`` such as a tuple or numpy object
        array of length ``A``.
    discount : float
        Discount factor. The per time-step discount factor on future rewards.
        Valid values are greater than 0 upto and including 1. If the discount
        factor is 1, then convergence is cannot be assumed and a warning will
        be displayed. Subclasses of ``MDP`` may pass ``None`` in the case where
        the algorithm does not use a discount factor.
    epsilon : float
        Stopping criterion. The maximum change in the value function at each
        iteration is compared against ``epsilon``. Once the change falls below
        this value, then the value function is considered to have converged to
        the optimal value function. Subclasses of ``MDP`` may pass ``None`` in
        the case where the algorithm does not use an epsilon-optimal stopping
        criterion.
    max_iter : int
        Maximum number of iterations. The algorithm will be terminated once
        this many iterations have elapsed. This must be greater than 0 if
        specified. Subclasses of ``MDP`` may pass ``None`` in the case where
        the algorithm does not use a maximum number of iterations.
    skip_check : bool
        By default we run a check on the ``transitions`` and ``rewards``
        arguments to make sure they describe a valid MDP. You can set this
        argument to True in order to skip this check.

    Attributes
    ----------
    P : array
        Transition probability matrices.
    R : array
        Reward vectors.
    V : tuple
        The optimal value function. Each element is a float corresponding to
        the expected value of being in that state assuming the optimal policy
        is followed.
    discount : float
        The discount rate on future rewards.
    max_iter : int
        The maximum number of iterations.
    policy : tuple
        The optimal policy.
    time : float
        The time used to converge to the optimal policy.
    verbose : boolean
        Whether verbose output should be displayed or not.

    Methods
    -------
    run
        Implemented in child classes as the main algorithm loop. Raises an
        exception if it has not been overridden.
    setSilent
        Turn the verbosity off
    setVerbose
        Turn the verbosity on

    """


In [7]:
#P = transtion
#if agent tries to move left in s1 there is a 10% probability 
#they will stay in s1 and 90% probability they will move to s2
#Agent is in       S1   S2    S3    S4
#left = array ([[  0.1   0.9   0.   0. ]
#                [ 0.1   0.    0.9  0. ]
#                [ 0.1   0.    0.   0.9]
#                [ 0.1   0.    0.   0.9]])

#Agent is in      S1    S2   S3  S4
#right =array ([[ 1.   0.   0.   0. ]
#               [ 1.   0.   0.   0. ]
#               [ 1.   0.   0.   0. ]
#               [ 1.   0.   0.   0. ]])

#R = reward
#enter S1 from anywhere get a reward of 0 
#S2 and S3 reward of 1 and S4 reward of 2
#[[ 0.  0.]
#[ 0.  1.]
#[ 0.  1.]
#[ 2.  2.]]


P, R = mdptoolbox.example.forest(4,2)
print (P)
print (R)

[[[ 0.1  0.9  0.   0. ]
  [ 0.1  0.   0.9  0. ]
  [ 0.1  0.   0.   0.9]
  [ 0.1  0.   0.   0.9]]

 [[ 1.   0.   0.   0. ]
  [ 1.   0.   0.   0. ]
  [ 1.   0.   0.   0. ]
  [ 1.   0.   0.   0. ]]]
[[ 0.  0.]
 [ 0.  1.]
 [ 0.  1.]
 [ 2.  2.]]


In [8]:
#policy iteration
pi = mdptoolbox.mdp.PolicyIteration(P, R, 0.9)
pi.run()

In [9]:
#cumulative reward
print (pi.V)
#optimal policy
print (pi.policy)
#max iterations
print (pi.iter)
#convergence time
print (pi.time)

(10.628820000000008, 11.941020000000009, 13.56102000000001, 15.561020000000008)
(0, 0, 0, 0)
3
0.003929853439331055


In [10]:
#value iteration
vi = mdptoolbox.mdp.ValueIteration(P, R, 0.9)
vi.run()

In [11]:
#cumulative reward
print (vi.V)
#optimal policy
print (vi.policy)
#max iterations
print (vi.iter)
#convergence time
print (vi.time)

(3.0151665990000005, 4.327366599, 5.947366599000001, 7.947366599000001)
(0, 0, 0, 0)
6
0.0006978511810302734


In [12]:
def value_iteration(discount_factor):
    global P, R
    P, R = mdptoolbox.example.forest(is_sparse=True) 
    global vi
    vi = mdptoolbox.mdp.ValueIteration(P,R,discount_factor)
    
    vi.run()
    
    global policy 
    policy = vi.policy
    global V 
    V = vi.V
    
    return policy, V

In [13]:
def policy_iteration(n_s, n_obs, discount_factor):
    #create transition probability matrix and reward matrix 
    
    global P, R, V
    P, R = mdptoolbox.example.rand(n_s,n_obs, V)
    
    #policy iteration using mdptoolbox
    #parameters : transiton probability matrix,
    #rewards matrix ,discount factor
    global pi
    pi = mdptoolbox.mdp.PolicyIteration(P, R, discount_factor)
    
    #run the policy iteration
    pi.run()
    
    global policy 
    policy = pi.policy
    V = pi.V
    print(V)
    
    return policy, V

In [14]:
rl_models = [
    ('Value Iteration', value_iteration(0.9)),
    ('Policy Iteration', policy_iteration(4, 2, 0.9))
 ]

(4.265663364171351, 4.132235342863298, 4.611430101402171, 4.296594026590925)


In [15]:
rl_models

[('Value Iteration',
  ((0, 0, 0), (5.051970000000001, 8.291970000000001, 12.291970000000001))),
 ('Policy Iteration',
  ((1, 1, 1, 1),
   (4.265663364171351,
    4.132235342863298,
    4.611430101402171,
    4.296594026590925)))]